In [1]:
#imports
import numpy as np
import pandas as pd
from hyperopt import hp
from hyperopt.pyll import scope
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPRegressor

import os
import sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from models.hyperopt_model import HyperoptModel

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
# read the data
preprocessed_data = pd.read_pickle('../data/preprocessed_science_quantitative.pd')

# Create a dataframe
df = pd.DataFrame(preprocessed_data)

In [3]:
# Create two new dataframes, one with the training rows, one with the test rows
train, test=train_test_split(df, test_size=0.2, random_state=0)

The history saving thread hit an unexpected error (OperationalError('database is locked',)).History will not be written to the database.


In [4]:
# Show the number of observations for the test and training dataframes
print('Number of observations in the training data:', len(train))
print('Number of observations in the test data:',len(test))

Number of observations in the training data: 79342
Number of observations in the test data: 19836


In [5]:
    #some parameters
    DO_LOWESS = False
    hyperopt = True
    # Run MLPRegressor with hyperopt optimization
    nn = HyperoptModel(train.copy(), test.copy(),'nn', cv=3, max_evals=500)
    features = list(df)
    features.remove('score')
    nn.raw_features = features

    nn.pipeline = Pipeline([
        ('estimate', MLPRegressor())
    ])
   
    nn.space = {
         'estimate__alpha' : hp.uniform('estimate__alpha', 0.001, 1),
         'estimate__activation' : hp.choice('estimate__activation', ['logistic']), # 'identity', 'logistic', 'tanh', 'relu'
         #'estimate__learning_rate' : hp.choice('estimate__learning_rate', ['constant', 'invscaling', 'adaptive']),
         'estimate__hidden_layer_sizes' : scope.int(hp.uniform('estimate__hidden_layer_sizes', 1, 100)),
         'estimate__solver' : hp.choice('estimate__solver', ['adam']), #'lbfgs', 'sgd',
         #'estimate__max_iter' : scope.int(hp.uniform('estimate__max_iter', 500, 1000))
    }

    if hyperopt:
        nn.run(do_lowess=DO_LOWESS)
    else:
        # train with default params
        nn.pipeline.fit(X=nn.X_train, y=nn.y_train)
        nn.model = cat.pipeline
        nn.stats()
        nn.plot_predicted_vs_actual(do_lowess=DO_LOWESS)
        nn.plot_feature_importance()
        nn.qq_plot()

Performing parameters optimization...
[1/500]	cv_eval_time=150.78 sec	RMSE=0.098939	R^2=0.756994
[2/500]	cv_eval_time=132.38 sec	RMSE=0.101303	R^2=0.753978
[3/500]	cv_eval_time=70.64 sec	RMSE=0.095592	R^2=0.773527
[4/500]	cv_eval_time=97.59 sec	RMSE=0.098261	R^2=0.761928
[5/500]	cv_eval_time=68.48 sec	RMSE=0.108023	R^2=0.708241
[6/500]	cv_eval_time=33.23 sec	RMSE=0.103762	R^2=0.731508
[7/500]	cv_eval_time=39.66 sec	RMSE=0.099922	R^2=0.753427
[8/500]	cv_eval_time=37.58 sec	RMSE=0.104366	R^2=0.730330
[9/500]	cv_eval_time=30.19 sec	RMSE=0.108095	R^2=0.708643
[10/500]	cv_eval_time=47.61 sec	RMSE=0.097412	R^2=0.767029
[11/500]	cv_eval_time=28.07 sec	RMSE=0.098787	R^2=0.756641
[12/500]	cv_eval_time=32.82 sec	RMSE=0.103561	R^2=0.730562
[13/500]	cv_eval_time=26.02 sec	RMSE=0.109772	R^2=0.687763
[14/500]	cv_eval_time=40.58 sec	RMSE=0.112476	R^2=0.691447
[15/500]	cv_eval_time=28.31 sec	RMSE=0.107479	R^2=0.703725
[16/500]	cv_eval_time=32.60 sec	RMSE=0.103022	R^2=0.736746
[17/500]	cv_eval_time=21.

[139/500]	cv_eval_time=28.23 sec	RMSE=0.096048	R^2=0.769880
[140/500]	cv_eval_time=20.14 sec	RMSE=0.098228	R^2=0.761100
[141/500]	cv_eval_time=25.49 sec	RMSE=0.097880	R^2=0.764619
[142/500]	cv_eval_time=20.60 sec	RMSE=0.096334	R^2=0.767773
[143/500]	cv_eval_time=17.28 sec	RMSE=0.106524	R^2=0.713994
[144/500]	cv_eval_time=24.56 sec	RMSE=0.096688	R^2=0.765025
[145/500]	cv_eval_time=21.92 sec	RMSE=0.103576	R^2=0.731282
[146/500]	cv_eval_time=29.58 sec	RMSE=0.095222	R^2=0.770160
[147/500]	cv_eval_time=21.37 sec	RMSE=0.095166	R^2=0.775458
[148/500]	cv_eval_time=21.80 sec	RMSE=0.095802	R^2=0.775201
[149/500]	cv_eval_time=23.71 sec	RMSE=0.096081	R^2=0.771949
[150/500]	cv_eval_time=24.10 sec	RMSE=0.096768	R^2=0.769162
[151/500]	cv_eval_time=15.56 sec	RMSE=0.095752	R^2=0.772046
[152/500]	cv_eval_time=19.92 sec	RMSE=0.094854	R^2=0.775630
[153/500]	cv_eval_time=31.30 sec	RMSE=0.095724	R^2=0.772325
[154/500]	cv_eval_time=29.23 sec	RMSE=0.096927	R^2=0.767433
[155/500]	cv_eval_time=27.28 sec	RMSE=0.

[276/500]	cv_eval_time=19.65 sec	RMSE=0.096648	R^2=0.765906
[277/500]	cv_eval_time=23.59 sec	RMSE=0.096249	R^2=0.766754
[278/500]	cv_eval_time=26.13 sec	RMSE=0.095815	R^2=0.772357
[279/500]	cv_eval_time=24.02 sec	RMSE=0.096726	R^2=0.767613
[280/500]	cv_eval_time=26.41 sec	RMSE=0.096306	R^2=0.769678
[281/500]	cv_eval_time=23.78 sec	RMSE=0.096979	R^2=0.764486
[282/500]	cv_eval_time=15.92 sec	RMSE=0.095345	R^2=0.770380
[283/500]	cv_eval_time=18.21 sec	RMSE=0.096478	R^2=0.771129
[284/500]	cv_eval_time=22.48 sec	RMSE=0.101120	R^2=0.762141
[285/500]	cv_eval_time=17.66 sec	RMSE=0.094719	R^2=0.774725
[286/500]	cv_eval_time=15.65 sec	RMSE=0.095984	R^2=0.771324
[287/500]	cv_eval_time=23.59 sec	RMSE=0.097592	R^2=0.762605
[288/500]	cv_eval_time=16.64 sec	RMSE=0.096892	R^2=0.766704
[289/500]	cv_eval_time=24.40 sec	RMSE=0.097108	R^2=0.772077
[290/500]	cv_eval_time=23.40 sec	RMSE=0.096459	R^2=0.770040
[291/500]	cv_eval_time=19.08 sec	RMSE=0.109827	R^2=0.708244
[292/500]	cv_eval_time=22.61 sec	RMSE=0.

[413/500]	cv_eval_time=17.08 sec	RMSE=0.094768	R^2=0.775337
[414/500]	cv_eval_time=24.13 sec	RMSE=0.095497	R^2=0.764787
[415/500]	cv_eval_time=23.48 sec	RMSE=0.096195	R^2=0.770562
[416/500]	cv_eval_time=23.44 sec	RMSE=0.096636	R^2=0.765756
[417/500]	cv_eval_time=24.37 sec	RMSE=0.097124	R^2=0.771055
[418/500]	cv_eval_time=16.33 sec	RMSE=0.094855	R^2=0.772737
[419/500]	cv_eval_time=23.82 sec	RMSE=0.096702	R^2=0.767913
[420/500]	cv_eval_time=27.03 sec	RMSE=0.096273	R^2=0.768793
[421/500]	cv_eval_time=20.38 sec	RMSE=0.095307	R^2=0.772124
[422/500]	cv_eval_time=23.58 sec	RMSE=0.096350	R^2=0.768578
[423/500]	cv_eval_time=15.81 sec	RMSE=0.095088	R^2=0.775901
[424/500]	cv_eval_time=24.02 sec	RMSE=0.097134	R^2=0.767729
[425/500]	cv_eval_time=20.68 sec	RMSE=0.097235	R^2=0.762047
[426/500]	cv_eval_time=24.37 sec	RMSE=0.095613	R^2=0.769883
[427/500]	cv_eval_time=22.36 sec	RMSE=0.095401	R^2=0.774384
[428/500]	cv_eval_time=23.02 sec	RMSE=0.096904	R^2=0.766380
[429/500]	cv_eval_time=19.81 sec	RMSE=0.